<a href="https://colab.research.google.com/github/igor531205/nlp/blob/main/home_work_3/Fine_tuning_model_for_question_answering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Выполнил Пушкарев Игорь Игоревич. Группа 23.М08-мм.***

### Sentence Transformers.

*Установим Transformers и Datasets из Hugging Face*

In [1]:
!pip install -q sentence-transformers transformers[torch] datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 17.2 MB/s eta 0:00:00


*Импортируем необходимые для дальнейшей работы библиотеки.*

In [2]:
import torch
from tqdm import tqdm
from datasets import load_dataset
from transformers import DefaultDataCollator, TrainingArguments, Trainer, \
                         AutoTokenizer, AutoModelForQuestionAnswering

*Загрузим Dataset [sberquad](https://huggingface.co/datasets/sberquad)*

In [3]:
# Load the dataset
sberquad = load_dataset('sberquad')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/45328 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5036 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/23936 [00:00<?, ? examples/s]

*Выведем содержимое датасета*

In [4]:
sberquad

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 45328
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5036
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 23936
    })
})

*Выведем 1 вопрос-ответ из датасета*

In [5]:
example = sberquad['train'][11]
for key in example:
    print(key, ':', example[key])

id : 18340
title : SberChallenge
context : Город Байконур и космодром Байконур вместе образуют комплекс Байконур , арендованный Россией у Казахстана на период до 2050 года. Эксплуатация космодрома стоит около 9 млрд рублей в год (стоимость аренды комплекса Байконур составляет 115 млн долларов — около 7,4 млрд рублей в год; ещё около 1,5 млрд рублей в год Россия тратит на поддержание объектов космодрома), что составляет 4,2 % от общего бюджета Роскосмоса на 2012 год. Кроме того, из федерального бюджета России в бюджет города Байконура ежегодно осуществляется безвозмездное поступление в размере 1,16 млрд рублей (по состоянию на 2012 год). В общей сложности космодром и город обходятся бюджету России в 10,16 млрд рублей в год.
question : Сколько стоит эксплуатация космодрома?
answers : {'text': ['9 млрд рублей в год'], 'answer_start': [166]}


*Загрузим Sentence-Transformers Model [ai-forever/ruBert-base](https://huggingface.co/ai-forever/ruBert-base)*

In [6]:
# Load the tokenizer and model
model_name = 'ai-forever/ruBert-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/590 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/716M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at ai-forever/ruBert-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


*Токенизируем данные с помощью токенизатора выбранной модели.*

In [7]:
def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride.
    # This results in one example possible giving several features when a context is long,
    # each of those features having a context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples['question'],
        examples['context'],
        truncation='only_second',  # truncate context, not the question
        max_length=384,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding='max_length',
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop('overflow_to_sample_mapping')
    # The offset mappings will give us a map from token to character position in the original context.
    # This will help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop('offset_mapping')

    # Let's label those examples!
    tokenized_examples['start_positions'] = []
    tokenized_examples['end_positions'] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples['input_ids'][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples['answers'][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers['answer_start']) == 0:
            tokenized_examples['start_positions'].append(cls_index)
            tokenized_examples['end_positions'].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers['answer_start'][0]
            end_char = start_char + len(answers['text'][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples['start_positions'].append(cls_index)
                tokenized_examples['end_positions'].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples['start_positions'].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples['end_positions'].append(token_end_index + 1)

    return tokenized_examples

*Применим функцию токенизации к данным*

In [8]:
# Apply the function to our data
tokenized_datasets = sberquad.map(prepare_train_features, batched=True, remove_columns=sberquad['train'].column_names)

Map:   0%|          | 0/45328 [00:00<?, ? examples/s]

Map:   0%|          | 0/5036 [00:00<?, ? examples/s]

Map:   0%|          | 0/23936 [00:00<?, ? examples/s]

*Выведем содержимое токенизированного датасета*

In [9]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 45691
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 5090
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 24113
    })
})

*Подготовим аргументы для обучения модели*

In [10]:
args = TrainingArguments(
    f'finetune-ruBERT-sberquad',
    evaluation_strategy = 'epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

*Подготовим модель для обучения*

In [11]:
data_collator = DefaultDataCollator()

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

*Обучим модель*

In [12]:
# Run the trainer
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.608600,1.510303
2,1.301400,1.450782
3,1.057800,1.535958


TrainOutput(global_step=8568, training_loss=1.4137174159316233, metrics={'train_runtime': 11593.2678, 'train_samples_per_second': 11.823, 'train_steps_per_second': 0.739, 'total_flos': 2.6862547752055296e+16, 'train_loss': 1.4137174159316233, 'epoch': 3.0})

*Сохраним модель*

In [13]:
# Save the model and tokenizer
trainer.save_model('model_qa')

*Подготовим функцию ответа на вопрос*

In [14]:
def predicted_answer(instance):
    # Question and context
    context = instance['context']
    question = instance['question']

    # Tokenize the data
    inputs = tokenizer(question, context, return_tensors='pt', max_length=512, truncation=True)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Get model's output
    with torch.no_grad():
        output = model(**inputs)

    # Get the predicted answer
    start_idx = torch.argmax(output.start_logits)
    end_idx = torch.argmax(output.end_logits)

    predicted_answer = tokenizer.convert_tokens_to_string(\
                      tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][start_idx:end_idx + 1])
    )

    print('Вопрос:', question)
    print('Ответ:', predicted_answer)

*Проверим на тестовом датасете*

In [50]:
predicted_answer(sberquad['test'][1])

Вопрос: Какие животные появились на Земле 2,1 миллиарда лет назад?
Ответ: многоклеточные


In [17]:
predicted_answer(sberquad['test'][3])

Вопрос: Как называется внесистематическая категория живых организмов, тело которых состоит из многих клеток, большая часть которых дифференцированы?
Ответ: многоклеточныи организм


In [56]:
predicted_answer(sberquad['test'][7])

Вопрос: Что означает дифференцированы?
Ответ: различаются по строению и выполняемым функциям


In [18]:
predicted_answer(sberquad['test'][11])

Вопрос: Многоклеточный организм — внесистематическая категория каких организмов?
Ответ: живых организмов


In [52]:
predicted_answer(sberquad['test'][33])

Вопрос: Как называется традиционное фламандское поселение?
Ответ: традиционное поселение — хутор.


*Проверим качество ответа на train данных*

In [35]:
instance = sberquad['train'][1]
context = instance['context']
question = instance['question']

given_answer = instance['answers']['text'][0]  # Assuming the first answer is the correct one
given_answer_start = instance['answers']['answer_start'][0]

# Tokenize the data
inputs = tokenizer(question, context, return_tensors='pt', max_length=512, truncation=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

inputs = {k: v.to(device) for k, v in inputs.items()}

# Get model's output
with torch.no_grad():
    output = model(**inputs)

# Get the predicted answer
start_idx = torch.argmax(output.start_logits)
end_idx = torch.argmax(output.end_logits)

predicted_answer = tokenizer.convert_tokens_to_string(\
    tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][start_idx:end_idx + 1])
)
print('Описание:', context)
print('Вопрос:', question)
print('Ответ из датасета:', predicted_answer)
print('Ответ модели:', given_answer)

Описание: В протерозойских отложениях органические остатки встречаются намного чаще, чем в архейских. Они представлены известковыми выделениями сине-зелёных водорослей, ходами червей, остатками кишечнополостных. Кроме известковых водорослей, к числу древнейших растительных остатков относятся скопления графито-углистого вещества, образовавшегося в результате разложения Corycium enigmaticum. В кремнистых сланцах железорудной формации Канады найдены нитевидные водоросли, грибные нити и формы, близкие современным кокколитофоридам. В железистых кварцитах Северной Америки и Сибири обнаружены железистые продукты жизнедеятельности бактерий.
Вопрос: что найдено в кремнистых сланцах железорудной формации Канады?
Ответ из датасета: нитевидные водоросли, грибные нити и формы
Ответ модели: нитевидные водоросли, грибные нити


*Подготовим функцию для оценки*

In [39]:
# Function to evaluate a single instance
def evaluate_instance(instance, device):
    context = instance['context']
    question = instance['question']
    given_answer = instance['answers']['text'][0]  # Assuming the first answer is the correct one

    # Tokenize the data
    inputs = tokenizer(question, context, return_tensors='pt', max_length=512, truncation=True)

    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Apply the BERT model
    with torch.no_grad():  # No need to calculate gradients
        output = model(**inputs)

    # Get the predicted answer
    start_idx = torch.argmax(output.start_logits)
    end_idx = torch.argmax(output.end_logits)
    predicted_answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][start_idx:end_idx + 1]))

    return predicted_answer.lower() == given_answer.lower()

*Оценим ответы первых 10000 экземпляров в train датасете*

In [48]:
# Evaluate the a number of instances
correct_count = 0
total_count = 10000

for i in tqdm(range(total_count)):
    correct_count += evaluate_instance(sberquad['train'][i], device)

100%|██████████| 10000/10000 [03:39<00:00, 45.64it/s]


*Посчитаем Accuracy модели на train датасете*

In [49]:
# Calculate and output the accuracy
accuracy = correct_count / total_count
print(f'Accuracy train: {accuracy * 100:.2f}%')

Accuracy train: 38.29%


*Оценим ответы первых 5000 экземпляров в validation датасете*

In [53]:
# Evaluate the a number of instances
correct_count = 0
total_count = 5000

for i in tqdm(range(total_count)):
    correct_count += evaluate_instance(sberquad['validation'][i], device)

100%|██████████| 5000/5000 [01:31<00:00, 54.77it/s]


*Посчитаем Accuracy модели на validation датасете*

In [54]:
# Calculate and output the accuracy
accuracy = correct_count / total_count
print(f'Accuracy validation: {accuracy * 100:.2f}%')

Accuracy validation: 32.22%
